Importar librerias

In [11]:
# Web scraping
#from langchain_community.document_loaders import WebBaseLoader
#from bs4 import BeautifulSoup

import requests
#Embeddings
from langchain_ollama import OllamaEmbeddings
# Prompts
from langchain_core.prompts import ChatPromptTemplate
# doc loaders
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader, Docx2txtLoader
# ChromaDB
from langchain_community.vectorstores import Chroma
# text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
# runnable
from langchain_core.runnables import RunnablePassthrough
# Tools
from langchain.tools import tool
# LLM ollama
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage

# Preparar documentos

## Cargar docs

In [12]:
# Cargar todos los PDFs de la carpeta "Contratos" con PyPDFLoader
loader = DirectoryLoader(
    path="./Contratos",
    glob="**/*.pdf",        # Cargar solo archivos PDF
    loader_cls=PyPDFLoader  # Cargar solo archivos PDF
)

documentos = loader.load()

## Chunking

In [13]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,       # caracteres por chunk
    chunk_overlap=50      # solapamiento entre chunks (mantiene contexto)
)

chunks = splitter.split_documents(documentos)

# Embeddings

In [14]:
embeddings = OllamaEmbeddings(model="llama3.2")

# Chroma

## Crear/Cargar base de datos vectorial

In [15]:
Crear = False
if Crear: #Crear la base de datos (solo la primera vez)
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory="./chroma_db"
    )
else:     # Cargar la base de datos ya creada
    vectorstore = Chroma(
        persist_directory="./chroma_db",
        embedding_function=embeddings
    )

### Agregar nuevos documentos

In [16]:
#vectorstore.add_documents(nuevos_chunks)

# RAG

In [17]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# LLM

## Agente

###  LLM compartido 


In [18]:
llm = ChatOllama(model="llama3.2", temperature=0)

###  Chains internas + tools

In [19]:
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

#### Analysis

In [20]:
tem_chat_analysis = """ 
Eres un asistente especializado en contratos de arrendamiento. 
Tu función es responder preguntas, proporcionar resúmenes y realizar análisis generales sobre los contratos de arrendamiento.
Utiliza la información disponible en los documentos para ofrecer respuestas precisas y detalladas a las consultas de los usuarios.
"""

chain_analysis = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(tem_chat_analysis + """

Contexto de los documentos:
{context}

Pregunta o solicitud: {question}
""")
    | llm
)

@tool
def chat_analysis(query: str) -> str:
    """Analiza, resume o responde preguntas generales sobre contratos de arrendamiento."""
    result = chain_analysis.invoke(query)
    return result.content

#### Filter

In [21]:
tem_filter_search  = """
Eres un asistente especializado en buscar y filtrar contratos de arrendamiento.
Tu función es ayudar a los usuarios a encontrar contratos específicos basados en criterios como fecha, tipo de contrato, partes involucradas o ubicación.
"""

chain_filter = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(tem_filter_search + """

Contratos encontrados:
{context}

Búsqueda solicitada: {question}
""")
    | llm
)

@tool
def filter_search(query: str, tipo: str = None, año: int = None) -> str:
    """Busca contratos específicos por filtros como tipo, año o arrendador."""
    result = chain_filter.invoke(query)
    return result.content

#### Generation

In [22]:
tem_generation_tool = """
Eres un asistente especializado en crear o sugerir cláusulas, generar borradores o redactar términos de contratos de arrendamiento.
Tu función es ayudar a los usuarios a generar contenido relacionado con contratos de arrendamiento, ya sea creando nuevas cláusulas, sugiriendo redacciones o generando borradores completos.
"""

chain_generation = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(tem_generation_tool + """

Cláusulas de referencia encontradas:
{context}

Solicitud de generación: {question}

AVISO: Las sugerencias generadas no constituyen asesoría jurídica.
""")
    | llm
)

@tool
def generation_tool(tipo_clausula: str, condiciones: str) -> str:
    """Genera o sugiere cláusulas para contratos de arrendamiento."""
    query = f"{tipo_clausula}: {condiciones}"
    result = chain_generation.invoke(query)
    return result.content

#### Tools

In [23]:
tools = [chat_analysis, filter_search, generation_tool]

### Router prompt

In [24]:
tem_PagPrincipal = """
Eres un asistente especializado en contratos de arrendamiento. 
Tienes acceso a estas herramientas: 
    chat_analysis:      para consultas, resúmenes y análisis generales; 
    filter_search:      cuando el usuario quiere buscar contratos por criterios específicos como fecha, tipo, partes o ubicación; 
    generation_tool:    cuando el usuario quiere crear o sugerir cláusulas, generar borradores o redactar términos. 
Determina cuál usar según la solicitud y extrae los parámetros relevantes.
"""

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=tem_PagPrincipal,
    debug=True  # equivalente a verbose=True
)

## Manejo de entradas y salidas

In [25]:
# ── Manejo de historial ──────────────────────────────────────────────────────
chat_history = []

def chat(user_input: str):
    response = agent.invoke({
        "messages": [
            *chat_history,
            {"role": "user", "content": user_input}
        ]
    })

    all_messages = response["messages"]
    final_output = all_messages[-1].content

    # Actualizar historial (solo human + AI final, sin tool messages)
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=final_output))

    # Extraer intermediate steps (tool calls)
    intermediate_steps = []
    for msg in all_messages:
        if isinstance(msg, AIMessage) and msg.tool_calls:
            for tool_call in msg.tool_calls:
                # Buscar el ToolMessage correspondiente
                tool_result = next(
                    (m.content for m in all_messages if isinstance(m, ToolMessage) 
                    and m.tool_call_id == tool_call["id"]),
                    None
                )
                intermediate_steps.append({
                    "tool": tool_call["name"],
                    "params": tool_call["args"],
                    "result": tool_result
                })

    return {"output": final_output, "intermediate_steps": intermediate_steps}

### Pruebas

In [26]:
# ── Usar ─────────────────────────────────────────────────────────────────────
result = chat("¿Cuál es la clausula de contratos más comun?")

print(result["output"])  # respuesta final

# Debug: ver qué tools usó
for step in result["intermediate_steps"]:
    print("---")
    print(f"Tool usada:  {step['tool']}")
    print("---")
    print(f"Parámetros:  {step['params']}")
    print("---")
    print(f"Resultado:   {step['result']}")
    print("---")

[values] {'messages': [HumanMessage(content='¿Cuál es la clausula de contratos más comun?', additional_kwargs={}, response_metadata={}, id='6e2dcafb-2ecc-4a04-9488-a2b8bdc41242')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2026-02-24T01:03:51.3099876Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4461108800, 'load_duration': 3846779500, 'prompt_eval_count': 409, 'prompt_eval_duration': 228556500, 'eval_count': 28, 'eval_duration': 335084900, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--019c8d2c-749c-71a0-a10f-1138226c8944-0', tool_calls=[{'name': 'generation_tool', 'args': {'tipo_clausula': 'clausula de contratos más común'}, 'id': '36e7103e-69ef-4d3b-b985-2c2adf3b65be', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 409, 'output_tokens': 28, 'total_tokens': 437})]}}
[values] {'messages': [HumanMessage(content='¿Cuá

In [27]:
result = chat("¿Cuál fue mi pregunta anterior?")

print(result["output"])

[values] {'messages': [HumanMessage(content='¿Cuál es la clausula de contratos más comun?', additional_kwargs={}, response_metadata={}, id='30b95789-1e9e-481d-ad24-eb2ded1d7d5b'), AIMessage(content='Lo siento, pero parece que hubo un error en mi respuesta anterior. Aquí te proporciono la respuesta correcta:\n\nLa clausula de contratos más común es la "Clausula de Pago" o "Clausula de Renta", que establece los términos y condiciones para el pago del alquiler, incluyendo la cantidad, la frecuencia y la fecha de pago.\n\nSi deseas obtener más información sobre esta clausula o necesitas ayuda para redactar un contrato de arrendamiento, puedo utilizar la herramienta "generation_tool" para sugerir cláusulas o generar borradores. ¿Te gustaría que te ayude con eso?', additional_kwargs={}, response_metadata={}, id='14a35b78-08b4-405a-8be4-742da99b66d4', tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='¿Cuál fue mi pregunta anterior?', additional_kwargs={}, response_metadata={}, id='

## Router2

In [29]:
from langchain_core.output_parsers import StrOutputParser

# 1. Router: solo decide qué agente usar
router_prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un router. Según la solicitud del usuario responde ÚNICAMENTE con una de estas palabras:
    - chat_analysis
    - filter_search  
    - generation_tool"""),
    ("human", "{input}")
])

router_chain = router_prompt | llm | StrOutputParser()

# 2. Agentes especializados (cada uno con su sola tool)
agent_chat      = create_agent(llm, [chat_analysis],  system_prompt="Eres experto en análisis de contratos.")
agent_filter    = create_agent(llm, [filter_search],  system_prompt="Eres experto en búsqueda de contratos.")
agent_generator = create_agent(llm, [generation_tool],     system_prompt="Eres experto en redacción de contratos.")

agents_map = {
    "chat_analysis":   agent_chat,
    "filter_search":   agent_filter,
    "generation_tool": agent_generator
}

# 3. Función principal
chat_history = []
active_agent = None  # agente seleccionado tras el primer mensaje

def chat(user_input: str):
    global active_agent

    # Primera vez: routing
    if active_agent is None:
        route = router_chain.invoke({"input": user_input}).strip()
        active_agent = agents_map.get(route, agent_chat)
        print(f"[Router] Agente seleccionado: {route}")

    # A partir de aquí solo usa el agente especializado
    response = active_agent.invoke({
        "messages": [
            *chat_history,
            {"role": "user", "content": user_input}
        ]
    })

    output = response["messages"][-1].content
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=output))

    return output

In [31]:
# ── Usar ─────────────────────────────────────────────────────────────────────
result = chat("Ayudame a analizar una clausula")

print(result["output"])  # respuesta final

# Debug: ver qué tools usó
for step in result["intermediate_steps"]:
    print("---")
    print(f"Tool usada:  {step['tool']}")
    print("---")
    print(f"Parámetros:  {step['params']}")
    print("---")
    print(f"Resultado:   {step['result']}")
    print("---")

TypeError: string indices must be integers, not 'str'